# AI-driven audio enhancement for your audio files, powered by [Resemble AI](https://github.com/resemble-ai/resemble-enhance)
We're going to use this library to clean up the female voice of the [Wolof Audio TTS](https://huggingface.co/datasets/galsenai/wolof_tts) dataset to remove microphone background noise and boost the volume of the audios.
> This notebook is made by [Derguene](https://github.com/DerXter)

## Intalling the library + dependencies

In [ ]:
!pip install soundfile
!git clone https://github.com/resemble-ai/resemble-enhance.git
!pip install -e resemble-enhance/

## Denoising and Enhancement

In [ ]:
from resemble_enhance.enhancer.inference import denoise, enhance
import torch
import torchaudio

if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

### Parameters
* __NFE__ = Number of Function Evaluations (higher values in general yield better quality but may be slower)
* __TAU__ = Prior Temperature (higher values can improve quality but can reduce stability)
* __Solver__ = CFM ODE Solver (Midpoint is recommended)
> __CFM__ = Conditional Flow Matching model

In [ ]:
def denoise_and_enhance(path, solver, nfe, tau, denoising):
    """ return (sample rate, denoised
    """

    solver = solver.lower()
    nfe    = int(nfe)
    lambd  = 0.9 if denoising else 0.1

    dwav, sr = torchaudio.load(path)
    dwav     = dwav.mean(dim=0)

    wav1, new_sr = denoise(dwav, sr, device)
    wav2, new_sr = enhance(dwav, sr, device, 
                           nfe=nfe, solver=solver, 
                           lambd=lambd, tau=tau)

    wav1 = wav1.cpu().numpy()
    wav2 = wav2.cpu().numpy()

    return (new_sr, wav1), (new_sr, wav2)

In [ ]:
from pathlib import Path

data_dir = "path/to/your/input/dir/"
audios = list(Path(data_dir).rglob(f'*.wav'))

print(f"There are {len(audios)} raw audio files!")

In [ ]:
import soundfile as sf
from tqdm import tqdm

for audio in audios:

    output = denoise_and_enhance(path  = audio,
                                 solver = "Midpoint",
                                 nfe    = 64,
                                 tau    = 0.7,
                                 denoising = True
                                 )

    output_dir  = 'path/to/your/output/dir/'
    file_name   = str(audio).split('/')[-1] # To be adapted
    audio_array = output[1][1]
    sample_rate = output[1][0]
    
    sf.write(output_dir+file_name, 
             audio_array, 
             sample_rate, 
             format='wav')

cleaned_audios = list(Path(output_dir).rglob(f'*.wav'))
print(f"{len(cleaned_audios)} audio files successfully cleaned")